<a href="https://colab.research.google.com/github/shu0518/Gen_AI_final_m11423036/blob/main/%E5%BB%BA%E7%AB%8B%E5%90%91%E9%87%8F%E8%B3%87%E6%96%99%E5%BA%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI 教授分身：基於 RAG 的課程智慧助教

### 1. 更新必要套件並引入

In [ ]:
!pip install -U langchain langchain-community pypdf pymupdf python-docx faiss-cpu unstructured gdown
!pip install -U sentence-transformers
!pip install langchain-text-splitters

In [ ]:
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader,PyPDFLoader, UnstructuredWordDocumentLoader, UnstructuredFileLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

print("✅ 套件載入完成")

### 2. 下載建立資料庫之資源

In [ ]:
import os
import shutil
from google.colab import drive

# 1. 設定統一的下載目標資料夾
download_folder = "uploaded_docs"

# 清空並重建目標資料夾
if os.path.exists(download_folder):
    shutil.rmtree(download_folder)
os.makedirs(download_folder, exist_ok=True)

print(f"📂 準備將檔案下載至: {download_folder}")

# ==========================================
# 🟢 Method 1: 從 Google Drive URL讀取 (預設啟用)
# 狀態: 讀取 PDF 簡報與 TXT 逐字稿
# 風險: 若短時間多人或重複下載，可能會觸發 Google 流量限制
# ==========================================
try:
    print("🚀 [Method 1] 正在嘗試從 Google Drive URL 下載...")

    # 【請修改這裡】貼上你的 Google Drive 資料夾連結
    # ⚠️ 務必確認權限已設為「知道連結的任何人都能檢視」
    folder_url = "https://drive.google.com/drive/folders/1BOtQjjP7cS6O1q5Dupc9Ga6_jk7tiKxG?usp=sharing"

    # --- 自動提取 ID 的小工具 (增加下載成功率) ---
    def get_id_from_url(url):
        match = re.search(r'/folders/([a-zA-Z0-9-_]+)', url)
        return match.group(1) if match else url

    folder_id = get_id_from_url(folder_url)

    # 使用 gdown 下載整個資料夾
    files = gdown.download_folder(id=folder_id, output=download_folder, quiet=False, use_cookies=False)

    if files:
        # 統計檔案
        pdf_count = len([f for f in os.listdir(download_folder) if f.endswith('.pdf')])
        txt_count = len([f for f in os.listdir(download_folder) if f.endswith('.txt')])

        print(f"\n✅ Google Drive 下載成功！共 {len(files)} 個檔案。")
        print(f"   (包含 {pdf_count} 份 PDF 簡報, {txt_count} 份 TXT 逐字稿)")
    else:
        print("⚠️ 下載失敗：資料夾似乎是空的，或權限未公開。")
        print("💡 建議切換使用下方 Method 2 (GitHub) 作為備案。")

except Exception as e:
    print(f"❌ Google Drive 下載錯誤: {e}")
    print("💡 可能原因：")
    print("1. 資料夾權限沒開成「公開」。")
    print("2. Google 流量限制 (Too many accesses)。")
    print("👉 若無法解決，請嘗試啟用下方的 GitHub 備案模式。")


# ==========================================
# 🟡 Method 2: 從 GitHub 下載 (備案 - 預設註解)
# 狀態: 僅讀取 TXT 逐字稿 (受限於上傳大小，無 PDF)
# ==========================================
"""
# ⚠️ 若要測試備案模式，請移除此區塊的註解符號 (三引號)，並註解掉上方 Method 1

print("\\n🚀 [Method 2] 正在切換使用 GitHub 下載模式 (備案)...")
github_repo_url = "https://github.com/你的帳號/你的倉庫名.git"

try:
    # 透過 git clone 下載
    result = os.system(f"git clone --depth 1 {github_repo_url} {download_folder}_tmp")

    if result == 0:
        # 移動檔案
        temp_dir = f"{download_folder}_tmp"
        for item in os.listdir(temp_dir):
            if item == ".git": continue
            s = os.path.join(temp_dir, item)
            d = os.path.join(download_folder, item)
            if os.path.isfile(s):
                shutil.copy2(s, d)

        shutil.rmtree(temp_dir)

        # 統計檔案類型
        files = os.listdir(download_folder)
        pdf_count = len([f for f in files if f.endswith('.pdf')])
        txt_count = len([f for f in files if f.endswith('.txt')])

        print(f"✅ GitHub 下載成功！共 {len(files)} 個檔案。")

        # ⚠️⚠️⚠️ 這裡是非常重要的「降級說明」 ⚠️⚠️⚠️
        print("⚠️【注意：此為輕量備案模式】")
        print(f"   - 目前僅載入 {txt_count} 份 TXT 逐字稿。")
        print(f"   - 由於 GitHub 單檔上傳限制 (25MB)，本來源【不包含】任何 PDF 簡報。")
        print("   - 知識庫將僅基於教授口語內容建立，缺少簡報上的視覺文字資訊。")
        print("   - 若需完整功能，請使用 Method 1 (Google Drive)。")

    else:
        print("⚠️ Git Clone 失敗，請檢查網址。")

except Exception as e:
    print(f"❌ GitHub 下載錯誤: {e}")
"""

# 最後檢查
final_files = os.listdir(download_folder)
if len(final_files) > 0:
    print(f"\n🎉 資料準備完成！共 {len(final_files)} 個檔案將進入向量化流程。")
else:
    print("\n❌ 警告：目標資料夾是空的，請檢查流程。")

In [ ]:
import gdown
import os
import shutil

# 1. 建立下載目標資料夾
download_folder = "uploaded_docs"

# 如果資料夾已存在，先清空 (避免重複下載或舊檔干擾)
if os.path.exists(download_folder):
    shutil.rmtree(download_folder)
os.makedirs(download_folder, exist_ok=True)

print(f"📂 準備將檔案下載至: {download_folder}")

# 2. 貼上放資料的雲端資料夾連結
folder_url = "https://drive.google.com/drive/folders/1BOtQjjP7cS6O1q5Dupc9Ga6_jk7tiKxG?usp=sharing"

# 3. 開始下載整個資料夾
print(f"🚀 開始從資料夾下載所有檔案...")

try:
    # gdown.download_folder 會自動抓取該資料夾內所有檔案
    files = gdown.download_folder(folder_url, output=download_folder, quiet=False, use_cookies=False)

    # 計算下載數量
    if files:
        print(f"\n🎉 下載作業結束！成功下載 {len(files)} 個檔案。")
        # 列出前幾個檔案確認
        print("下載的檔案範例:", os.listdir(download_folder)[:5])
    else:
        print("⚠️ 資料夾似乎是空的，或者下載失敗。請檢查連結權限是否為「公開」。")

except Exception as e:
    print(f"❌ 下載錯誤: {e}")
    print("💡 提示：請確認資料夾權限已開啟為「知道連結的任何人都能檢視」。")

### 3. 依 Google 建議加入 EmbeddingGemma 前綴

Google 建議, 在文本部份的 Embedding 要用以下格式:

    title: {title|none} | text: ...

而問題 Query 要用:

    Query：task: search result | query: ...

In [ ]:
class EmbeddingGemmaEmbeddings(HuggingFaceEmbeddings):
    def __init__(self, **kwargs):
        super().__init__(
            model_name="google/embeddinggemma-300m",   # HF 上的官方模型
            encode_kwargs={"normalize_embeddings": True},  # 一般檢索慣例
            **kwargs
        )

    def embed_documents(self, texts):
        # 文件向量：title 可用 "none"，或自行帶入檔名/章節標題以微幅加分
        texts = [f'title: none | text: {t}' for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        # 查詢向量：官方建議的 Retrieval-Query 前綴
        return super().embed_query(f'task: search result | query: {text}')

### 4. 載入文件

In [ ]:
folder_path = "uploaded_docs"  # 確保指向你的 uploaded_docs
documents = []

print("🚀 開始通用載入並處理檔案...")

# 定義副檔名與對應的 Loader
LOADER_MAPPING = {
    ".txt": (TextLoader, {"encoding": "utf-8"}),
    ".pdf": (PyMuPDFLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
}

# 確保資料夾存在，避免報錯
if not os.path.exists(folder_path):
    print(f"❌ 錯誤：找不到資料夾 {folder_path}，請確認第 2 步是否下載成功。")
else:
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        filename_no_ext = os.path.splitext(file)[0]
        ext = os.path.splitext(file)[1].lower()

        loaded_docs = []

        try:
            # 1. 選擇 Loader
            if ext in LOADER_MAPPING:
                loader_class, loader_args = LOADER_MAPPING[ext]
                loader = loader_class(file_path, **loader_args)
                print(f"📖 使用專用 Loader 讀取: {file}")
            else:
            # 2. 萬用備案
                print(f"🧐 使用通用 Loader (Unstructured) 嘗試讀取: {file}")
                loader = UnstructuredFileLoader(file_path)

            loaded_docs = loader.load()

        except Exception as e:
            print(f"⚠️ 無法讀取 {file}: {e}")
            continue

        # 3. 注入檔名 (Context Injection)
        for doc in loaded_docs:
            original_content = doc.page_content
            clean_content = original_content.replace("\n", " ").strip()

            # 過濾太短的內容
            if len(clean_content) < 10:
                continue

            # 格式化內容：加入來源標籤
            new_content = f"【課程來源: {filename_no_ext}】\n內容: {clean_content}"

            doc.page_content = new_content
            documents.append(doc)

    print(f"✅ 成功處理 {len(documents)} 個頁面片段")

### 5. 建立向量資料庫

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HuggingFace')

In [ ]:
from huggingface_hub import login
login(token=hf_token)

In [ ]:
# 1. 切分文本
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

print(f"✂️ 文件已切分為 {len(split_docs)} 個向量片段")

# 2. 實例化第 3 步定義的 Embedding 類別
try:
    print("ww 正在載入 Gemma Embedding 模型 ...")
    # 這裡會使用 Colab 的 GPU (如果有開啟)
    embedding_model = EmbeddingGemmaEmbeddings(
        model_kwargs={'device': 'cuda'}, # 強制嘗試使用 GPU 加速
        encode_kwargs={'normalize_embeddings': True}
    )
except Exception as e:
    print(f"⚠️ 模型載入失敗，請檢查是否已執行 HuggingFace Login: {e}")
    # 如果失敗，回退到 CPU
    embedding_model = EmbeddingGemmaEmbeddings()

# 3. 建立向量資料庫
print("🚀 正在將文字轉換為向量並建立索引...")
vectorstore = FAISS.from_documents(split_docs, embedding_model)

print("✅ 向量資料庫建立完成！(使用 Google Gemma 模型)")

### 6. 儲存向量資料庫

In [ ]:
vectorstore.save_local("faiss_db")

In [ ]:
!zip -r faiss_db.zip faiss_db
print("✅ 壓縮好的向量資料庫已儲存為 'faiss_db.zip'，請下載此檔案備份。")